In [44]:
import pandas as pd
import numpy as np

In [45]:
events_data = pd.read_csv("event_data_train.csv")
submission_data = pd.read_csv("submissions_data_train.csv")
events_data['date'] = pd.to_datetime(events_data.timestamp, unit = 's')
events_data['day'] = events_data.date.dt.date

submission_data['date'] = pd.to_datetime(submission_data.timestamp, unit = 's')
submission_data['day'] = submission_data.date.dt.date

users_data = events_data.groupby('user_id', as_index= False)\
        .agg({'timestamp': 'max'})\
        .rename({'timestamp':'last_timestamp'}, axis = 'columns')
now = 1526772811
drop_out_threasold = 2592000

users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threasold
users_scores = submission_data.pivot_table(index='user_id',\
                        columns = 'submission_status',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0)\
                        .reset_index()
users_data = users_data.merge(users_scores, on = 'user_id', how = 'outer')
users_data = users_data.fillna(0)
users_invent_data = events_data.pivot_table(index = 'user_id',
                        columns = 'action',
                        values  = 'step_id',
                        aggfunc = 'count',
                        fill_value = 0).reset_index()
users_data = users_data.merge(users_invent_data, how = 'outer')
users_days = events_data.groupby('user_id').day.nunique()
users_days.to_frame().reset_index()
users_data = users_data.merge(users_days, on = 'user_id', how = 'outer')
users_data['passed_corse'] = users_data.passed > 170

In [46]:
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_corse
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False


In [47]:
users_data[users_data.passed_corse== True].day.median()

20.0

In [48]:
events_data.head()

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15


In [49]:
user_min_time = events_data.groupby("user_id", as_index=False)\
    .agg({'timestamp':'min'})\
    .rename({'timestamp': 'min_timestamp'}, axis = 1)
users_data = users_data.merge(user_min_time, how = "outer")
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_corse,min_timestamp
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False,1472827464
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False,1514383364
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False,1434358476
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False,1466156809
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False,1521634660


In [50]:
events_data_train = pd.DataFrame()

In [51]:
# Wrong(slow) solution
#for user_id in users_data.user_id:
#    min_user_time = users_data[users_data.user_id == user_id].min_timestamp.item()
#    time_treshold = min_user_time + 3*24*60*60
#    user_events_data = events_data[(events_data.user_id == user_id) & (events_data.timestamp < time_treshold)]
#    events_data_train = events_data_train.append(user_events_data)

In [52]:
events_data['user_time'] = events_data.user_id.map(str) + "_" + events_data.timestamp.map(str)
events_data.head()

,step_id,timestamp,action,user_id,date,day,user_time
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895


In [53]:
learning_time_threshold =  3*24*60*60
user_min_time.head()

,user_id,min_timestamp
0,1,1472827464
1,2,1514383364
2,3,1434358476
3,5,1466156809
4,7,1521634660


In [54]:
user_learning_time_threshold = user_min_time.user_id.map(str) + "_" +(user_min_time.min_timestamp + learning_time_threshold).map(str)
user_min_time['user_learning_time_threshold'] = user_learning_time_threshold
events_data = events_data.merge(user_min_time[['user_id','user_learning_time_threshold']], how='outer')
events_data.head()

,step_id,timestamp,action,user_id,date,day,user_time,user_learning_time_threshold
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895,17632_1434600048
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895,17632_1434600048


In [55]:
events_data_train = events_data[events_data.user_time <= events_data.user_learning_time_threshold]

In [56]:
events_data_train.head()

,step_id,timestamp,action,user_id,date,day,user_time,user_learning_time_threshold
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895,17632_1434600048
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,17632_1434340895,17632_1434600048


In [57]:
submission_data['users_time'] = submission_data.user_id.map(str) + '_' + submission_data.timestamp.map(str)
submission_data = submission_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how='outer')
submissions_data_train = submission_data[submission_data.users_time <= submission_data.user_learning_time_threshold]
submissions_data_train.groupby('user_id').day.nunique().max()

4

In [58]:
X = submissions_data_train.groupby('user_id').day.nunique().to_frame().reset_index()\
    .rename(columns={
    'day':'days'
    })

In [59]:
steps_tried = submissions_data_train.groupby('user_id').step_id.nunique().to_frame().reset_index()\
    .rename(columns={
    'step_id': 'steps_tried'
    })
X = X.merge(steps_tried, how="outer", on='user_id')
X.head()

,user_id,days,steps_tried
0,2,1,2
1,3,1,4
2,8,1,11
3,14,1,1
4,16,3,23


In [60]:
X = X.merge(submissions_data_train.pivot_table(index='user_id',\
                        columns = 'submission_status',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0)\
                        .reset_index())

In [61]:
X['correct_ratio'] = X.correct/(X.correct + X.wrong)
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio
0,2,1,2,2,0,1.00
1,3,1,4,4,4,0.50
2,8,1,11,9,21,0.30
3,14,1,1,0,1,0.00
4,16,3,23,23,27,0.46


In [62]:
X = X.merge(events_data_train.pivot_table(index='user_id',\
                        columns = 'action',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0)\
                        .reset_index()[['user_id', 'viewed']],how='outer')

In [63]:
X = X.fillna(0)
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed
0,2,1.0,2.0,2.0,0.0,1.00,9
1,3,1.0,4.0,4.0,4.0,0.50,20
2,8,1.0,11.0,9.0,21.0,0.30,154
3,14,1.0,1.0,0.0,1.0,0.00,9
4,16,3.0,23.0,23.0,27.0,0.46,132


In [64]:
X = X.merge(users_data[['user_id','passed_corse','is_gone_user']], how = 'outer')

In [65]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_corse,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [66]:
X = X[~((X.is_gone_user == False) & (X.passed_corse == False))]

In [67]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_corse,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [68]:
y = X.passed_corse
X = X.drop(['passed_corse','is_gone_user'], axis = 1)

In [69]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed
0,2,1.0,2.0,2.0,0.0,1.00,9
1,3,1.0,4.0,4.0,4.0,0.50,20
2,8,1.0,11.0,9.0,21.0,0.30,154
3,14,1.0,1.0,0.0,1.0,0.00,9
4,16,3.0,23.0,23.0,27.0,0.46,132


In [74]:
X = X.set_index(X.user_id)
X= X.drop('user_id', axis = 1)

In [75]:
X.head()

,days,steps_tried,correct,wrong,correct_ratio,viewed
user_id,,,,,,
2,1.0,2.0,2.0,0.0,1.00,9
3,1.0,4.0,4.0,4.0,0.50,20
8,1.0,11.0,9.0,21.0,0.30,154
14,1.0,1.0,0.0,1.0,0.00,9
16,3.0,23.0,23.0,27.0,0.46,132


In [70]:
# Task
submissions = submission_data.drop(columns=['date', 'day'])
submissions = pd.get_dummies(submissions, columns=['submission_status'])

last_time = submissions.groupby('user_id').agg({ 'timestamp': 'max' })
last_time = last_time.rename(columns={ 'timestamp': 'last_timestamp' })

last_step_id = submissions.groupby('user_id').agg({ 'step_id': 'max' })
last_step_id = last_step_id.rename(columns={ 'step_id': 'last_step_id' })

submissions = submissions.merge(last_time, on='user_id')
submissions = submissions.merge(last_step_id, on='user_id')
filtered = submissions[
    (submissions.step_id == submissions.last_step_id) & \
    (submissions.submission_status_correct == 0)
]

filtered.groupby('step_id').agg({
    'user_id': 'count'
}).sort_values('user_id', ascending=False).head(10)

,user_id
step_id,
120745.0,1238
34041.0,527
33713.0,206
33540.0,198
33481.0,144
33439.0,102
33673.0,97
33539.0,77
31978.0,73


In [71]:
s_d = submission_data.drop(['date','day','user_id', 'timestamp'], axis=1)
s_d.head()

,step_id,submission_status,users_time,user_learning_time_threshold
0,31971.0,correct,15853_1434349275,15853_1434605177
1,31972.0,correct,15853_1434348300,15853_1434605177
2,31972.0,wrong,15853_1478852149,15853_1434605177
3,31972.0,correct,15853_1478852164,15853_1434605177
4,31976.0,wrong,15853_1434348123,15853_1434605177


In [72]:
s_d = s_d.loc[s_d.submission_status=='wrong'].step_id.value_counts()
s_d.head()

31978.0    16084
32031.0    13373
32202.0    13002
33481.0    10300
32075.0    10003
Name: step_id, dtype: int64

In [73]:
submission_data[submission_data['submission_status'] == 'wrong']\
                .groupby(['step_id'], as_index=False)\
                .agg({'submission_status': 'count'})\
                .sort_values(by='submission_status', ascending=False)\
                .step_id.iloc[0]

31978.0